<a href="https://colab.research.google.com/github/yzihan/Generative-AI/blob/main/notebooks/2%20Text-To-Video/1%20CLIP-Guided%20VQGAN%203D%20Turbo%20Zoom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![As Marvins conscious circuits began to shut down her silicon brain started dreaming](https://ipfs.pollinations.ai/ipfs/bafybeic3oqrdprhbnf4jwbt274f7mfxix6ljqll47hmqg2657l7runugae)

*As Marvins conscious circuits began to shut down her silicon brain started dreaming*

Words fail us, but a few of them and you have a 3D animation. Optional audio upload.

---
## Credits
The VQGAN+CLIP (z+quantize method) notebook this was based on is by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). The original BigGAN + CLIP method was made by https://twitter.com/advadnoun. Translated into Spanish and added explanations, and modifications by Eleiber#8347, and the friendly interface was made thanks to Abulafia#3734. Translated back into English, and zoom, pan, rotation, and keyframes features by Chigozie Nri (https://github.com/chigozienri, https://twitter.com/chigozienri). Some UI improvements were made by Justin John (https://github.com/justinjohn0306).[Somnai_dreams](https://twitter.com/somnai_dreams),

[thomash](https://twitter.com/pollinations_ai) added [gandamu_ml](https://twitter.com/gandamu_ml), and [zippy331](https://twitter.com/zippy731)'s 3D and turbo parts to this model.

In [ ]:
text_input = "As Marvins conscious circuits began to shut down her silicon brain started dreaming. Soviet realism." #@param {type:"string"}

# How strongly you want CLIP to move toward your prompt
step_size = 0.25 #@param {type:"number"}

frames =  150#@param {type:"number"}

fps = 10#@param {type:"number"}

width =  512#@param {type:"number"}
height =  512#@param {type:"number"}

# The camera preset to use
camera_preset = "follow_closest"  #@param ["only_complex_rotation","slow_complex_zoom_and_move","simple_forward_zoom_and_rotation","rotate_around_object", "follow_horizon", "follow_closest"]

# Optional audio file to use as input (uses WAV2CLIP to interpret the sound as images)
audio_file = "" #@param {type:"string"}

output_path = "/content"

In [ ]:
if audio_file == '':
  audio_file = None

# Zooming VQGAN+CLIP turbo 3D animations

This notebook allows you to create animations by providing text phrases, which are used by an AI to synthesise frames. You can provide values for how much each text phrase should be weighted, and how much zoom, pan, and translation there should be at each keyframe.

## Credits
The VQGAN+CLIP (z+quantize method) notebook this was based on is by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). The original BigGAN + CLIP method was made by https://twitter.com/advadnoun. Translated into Spanish and added explanations, and modifications by Eleiber#8347, and the friendly interface was made thanks to Abulafia#3734. Translated back into English, and zoom, pan, rotation, and keyframes features by Chigozie Nri (https://github.com/chigozienri, https://twitter.com/chigozienri). Some UI improvements were made by Justin John (https://github.com/justinjohn0306). A linked helper spreadsheet for creating parameter strings is by Kendrick Feller (https://twitter.com/EphemeralInc)

<h2>Donation Link</h2>

If you want, you can donate to support me at <a href="https://ko-fi.com/chigozie">ko-fi</a>. No pressure (but I'd love to know if you create something cool, or just had fun!).

<a href='https://ko-fi.com/chigozie' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://cdn.ko-fi.com/cdn/kofi2.png?v=3' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a>

<h2>Can I make NFTs?</h2>

If you must. If you do, consider using a low-energy blockchain such as Tezos, rather than Ethereum.

<h2>Too complicated? Want more?</h2>

If you want a custom animation, features not included here, or anything else, let me know. I might be up for accepting a commission!

In [ ]:
%cd /content

from tqdm import tqdm
import torch
import sys
from os.path import exists as path_exists
import pandas as pd

root_path = f'/content'
model_path = '/content/models' 

!git clone https://github.com/voodoohop/disco-diffusion.git
!cp disco-diffusion/disco_xform_utils.py disco_xform_utils.py
!git clone https://github.com/isl-org/MiDaS.git

# Rename a file to avoid a name conflict..
!mv MiDaS/utils.py MiDaS/midas_utils.py
sys.path.append('/content/MiDaS')

# Download and install adabins
!git clone https://github.com/shariqfarooq123/AdaBins.git
if not path_exists(f'{model_path}/AdaBins_nyu.pt'):
  !wget https://ipfs.pollinations.ai/ipfs/Qmd2mMnDLWePKmgfS8m6ntAg4nhV5VkUyAydYBp8cWWeB7/AdaBins_nyu.pt -P {model_path}
!mkdir pretrained
!cp  -P {model_path}/AdaBins_nyu.pt pretrained/AdaBins_nyu.pt
sys.path.append('/content/AdaBins')

# download and install pytorch3d-lite
!git clone https://github.com/MSFTserver/pytorch3d-lite
!cp pytorch3d-lite/py3d_tools.py .
sys.path.append('/content/pytorch3d-lite')

pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])

if not path_exists(f'{model_path}/dpt_large-midas-2f21e586.pt'):
  !wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt -P {model_path}

import disco_xform_utils as dxf
import py3d_tools as p3dT

In [ ]:
camera_presets = {
    "only_complex_rotation": {
        "translation_x": "0:(0)",
        "translation_y": "0:(0)",
        "translation_z": "0:(0)",
        "rotation_3d_x": "0:(0),10:(0.02),50:(0),75:(0.02),100:(0),125:(0.02),150:(0),175:(-0.02),200:(0),225:(0.01),250(0.02)",
        "rotation_3d_y": "0:(0)",
        "rotation_3d_z": "0:(0)",
    },
    "slow_complex_zoom_and_move": {
        "translation_x":"0:(0),22:(4.465),41:(0.355),61:(1.163),69:(-1.358),85:(0.079),107:(-0.843),116:(-4.123),136:(1.029),157:(1.074),166:(-3.439),187:(-0.214),209:(0.357),219:(-4.708),239:(0.49)",
        "translation_y":"0:(0),22:(2.42),41:(-0.019),61:(0.24),69:(-2.381),85:(-0.358),107:(0.097),116:(1.479),136:(0.425),157:(-0.401),166:(-2.366),187:(-0.508),209:(-0.525),219:(0.683),239:(0.351)",
        "translation_z":"0:(6)",
        "rotation_3d_x": "0:(0),22:(0.013),41:(-0.004),61:(-0.001),69:(-0.022),85:(0.005),107:(-0.002),116:(0.026),136:(0.004),157:(0.001),166:(0.027),187:(0.002),209:(-0.005),219:(-0.01),239:(-0.004)",
        "rotation_3d_y": "0:(0),21:(0.02),38:(0.001),53:(0.001),62:(0.016),82:(-0.004),102:(0.005),113:(0.012),130:(0.006),149:(0.002),159:(0.006),179:(0.005),200:(0.001),210:(-0.002),231:(0.005)",
        "rotation_3d_z": "0:(0),22:(0.007),41:(0.001),61:(0.005),69:(0.014),85:(-0.0),107:(-0.002),116:(0.028),136:(0.0),157:(0.003),166:(0.02),187:(-0.001),209:(-0.004),219:(-0.001),239:(-0.001)"
    },
    "simple_forward_zoom_and_rotation": {
        "translation_x": "0:(0)",
        "translation_y": "0:(0)",
        "translation_z": "0:(0),10:(20)",
        "rotation_3d_x": "0:(0),10:(-0.04)",
        "rotation_3d_y": "0:(0),20:(0.05)",
        "rotation_3d_z": "0:(0),30:(-0.01)",
    },
    "rotate_around_object": {
        "translation_x": "0:(-25)",
        "translation_y": "0:(0)",
        "translation_z": "0:(0)",
        "rotation_3d_x": "0:(0)",
        "rotation_3d_y": "0:(0.08)",
        "rotation_3d_z": "0:(0)"
    },
    "follow_horizon": {
        "translation_x":"0:(0),22:(4.465),41:(0.355),61:(1.163),69:(-1.358),85:(0.079),107:(-0.843),116:(-4.123),136:(1.029),157:(1.074),166:(-3.439),187:(-0.214),209:(0.357),219:(-4.708),239:(0.49)",
        "translation_y":"0:(0),22:(2.42),41:(-0.019),61:(0.24),69:(-2.381),85:(-0.358),107:(0.097),116:(1.479),136:(0.425),157:(-0.401),166:(-2.366),187:(-0.508),209:(-0.525),219:(0.683),239:(0.351)",
        "translation_z": "0:(8)",
        "rotation_3d_x": "0:(0)",
        "rotation_3d_y": "0:(0.08)",
        "rotation_3d_z": "0:(0)"
    },
    "follow_closest": {
        "translation_x":"0:(0),22:(4.465),41:(0.355),61:(1.163),69:(-1.358),85:(0.079),107:(-0.843),116:(-4.123),136:(1.029),157:(1.074),166:(-3.439),187:(-0.214),209:(0.357),219:(-4.708),239:(0.49)",
        "translation_y":"0:(0),22:(2.42),41:(-0.019),61:(0.24),69:(-2.381),85:(-0.358),107:(0.097),116:(1.479),136:(0.425),157:(-0.401),166:(-2.366),187:(-0.508),209:(-0.525),219:(0.683),239:(0.351)",
        "translation_z": "0:(8)",
        "rotation_3d_x": "0:(0)",
        "rotation_3d_y": "0:(0.08)",
        "rotation_3d_z": "0:(0)"
    },
}


In [ ]:

text_prompts = {
    0: text_input.split("|"),
    #140: ["dusty gymnasium with blue floor in the style high resolution medium format photograph"],
}



interval =  1#@param {type:"number"}
initial_image = "" #@param {type:"string"}
target_images = None #{  
#    0: ["/content/targets/out0158.jpg","/content/targets/out0108.jpg","/content/targets/out0092.jpg"]
#} 


seed = 1#@param {type:"number"}
#max_frames = 300#@param {type:"number"}

iterations_per_frame = {0: 20, 5: 10}



interp_spline = 'Linear' #Do not change, currently will not look good. param ['Linear','Quadratic','Cubic']{type:"string"}

# Unused
angle = "0:(0)"#@param {type:"string"}
zoom = "0: (1.05), 1: (1.07), 2: (1.16), 3: (1.43), 4: (1.43), 5: (1.09), 6: (1.21), 7: (1.81), 8: (1.72), 9: (1.16), 10: (1.22), 11: (1.24), 12: (1.10), 13: (1.11), 14: (1.09), 15: (1.08), 16: (1.09), 17: (1.13), 18: (1.10), 19: (1.09), 20: (1.16), 21: (1.15), 22: (1.19), 23: (1.09), 24: (1.56), 25: (1.29), 26: (1.10), 27: (1.15), 28: (1.69), 29: (1.65), 30: (1.20), 31: (1.21), 32: (1.19), 33: (1.30), 34: (1.07), 35: (1.13), 36: (1.08), 37: (1.08), 38: (1.11), 39: (1.12), 40: (1.10), 41: (1.08), 42: (1.18), 43: (1.17), 44: (1.10), 45: (1.21), 46: (1.33), 47: (1.37), 48: (1.08), 49: (1.21), 50: (1.56), 51: (1.84), 52: (1.16), 53: (1.20), 54: (1.37), 55: (1.09), 56: (1.12), 57: (1.08), 58: (1.08), 59: (1.09), 60: (1.13), 61: (1.08), 62: (1.08), 63: (1.14), 64: (1.22), 65: (1.14), 66: (1.11), 67: (1.41), 68: (1.44), 69: (1.07), 70: (1.17), 71: (2.05), 72: (1.50), 73: (1.17), 74: (1.24), 75: (1.22), 76: (1.26), 77: (1.09), 78: (1.11), 79: (1.08), 80: (1.08), 81: (1.12), 82: (1.10), 83: (1.11), 84: (1.09), 85: (1.22), 86: (1.16), 87: (1.10), 88: (1.28), 89: (1.31), 90: (1.25), 91: (1.13), 92: (1.27), 93: (1.63), 94: (1.60), 95: (1.16), 96: (1.17), 97: (1.37), 98: (1.07), 99: (1.12), 100: (1.07), 101: (1.08), 102: (1.10), 103: (1.13), 104: (1.12), 105: (1.08), 106: (1.13), 107: (1.16), 108: (1.13), 109: (1.11), 110: (1.38), 111: (1.47), 112: (1.07), 113: (1.18), 114: (1.95), 115: (1.67), 116: (1.13), 117: (1.24), 118: (1.20), 119: (1.20), 120: (1.07), 121: (1.10), 122: (1.09), 123: (1.09), 124: (1.13), 125: (1.12), 126: (1.09), 127: (1.10), 128: (1.21), 129: (1.18), 130: (1.08), 131: (1.38), 132: (1.25), 133: (1.22), 134: (1.11), 135: (1.30), 136: (1.64), 137: (1.40), 138: (1.18), 139: (1.13), 140: (1.37), 141: (1.08), 142: (1.14), 143: (1.08), 144: (1.08), 145: (1.10), 146: (1.12), 147: (1.08), 148: (1.09), 149: (1.13), 150: (1.22), 151: (1.13), 152: (1.14), 153: (1.47), 154: (1.56), 155: (1.07), 156: (1.20), 157: (1.93), 158: (1.80), 159: (1.13), 160: (1.26), 161: (1.17), 162: (1.22), 163: (1.09), 164: (1.10), 165: (1.08), 166: (1.08), 167: (1.12), 168: (1.11), 169: (1.10), 170: (1.12), 171: (1.14), 172: (1.15), 173: (1.09), 174: (1.06), 175: (1.07), 176: (1.08), 177: (1.11), 178: (1.20), 179: (1.17), 180: (1.11), 181: (1.12), 182: (1.15), 183: (1.12), 184: (1.08), 185: (1.11), 186: (1.06), 187: (1.06), 188: (1.28), 189: (1.62), 190: (1.11), 191: (1.07), 192: (1.09), 193: (1.07), 194: (1.07), 195: (1.07), 196: (1.07), 197: (1.07), 198: (1.09), 199: (1.16), 200: (1.16), 201: (1.15), 202: (1.08), 203: (1.20), 204: (1.14), 205: (1.08), 206: (1.08), 207: (1.08), 208: (1.06), 209: (1.12), 210: (1.52), 211: (1.35), 212: (1.06), 213: (1.08), 214: (1.08), 215: (1.06), 216: (1.08), 217: (1.06), 218: (1.07), 219: (1.08), 220: (1.13), 221: (1.22), 222: (1.16), 223: (1.13), 224: (1.15), 225: (1.17), 226: (1.11), 227: (1.07), 228: (1.09), 229: (1.06), 230: (1.06), 231: (1.42), 232: (1.54), 233: (1.10), 234: (1.07), 235: (1.09), 236: (1.08), 237: (1.07), 238: (1.08), 239: (1.07), 240: (1.07), 241: (1.08), 242: (1.21), 243: (1.15), 244: (1.15), 245: (1.09), 246: (1.20), 247: (1.14), 248: (1.09), 249: (1.07), 250: (1.08), 251: (1.06), 252: (1.12), 253: (1.51), 254: (1.24), 255: (1.07), 256: (1.08), 257: (1.09), 258: (1.07), 259: (1.10), 260: (1.38), 261: (1.61), 262: (1.09), 263: (1.15), 264: (1.76), 265: (1.55), 266: (1.24), 267: (1.21), 268: (1.20), 269: (1.27), 270: (1.08), 271: (1.13), 272: (1.08), 273: (1.09), 274: (1.15), 275: (1.13), 276: (1.10), 277: (1.09), 278: (1.22), 279: (1.24), 280: (1.10), 281: (1.14), 282: (1.27), 283: (1.44), 284: (1.08), 285: (1.20), 286: (1.72), 287: (1.86), 288: (1.18), 289: (1.19), 290: (1.29), 291: (1.11), 292: (1.10), 293: (1.09), 294: (1.08), 295: (1.09), 296: (1.12), 297: (1.09), 298: (1.08), 299: (1.13), 300: (1.12), 301: (1.11), 302: (1.10), 303: (1.47), 304: (1.59), 305: (1.10), 306: (1.16), 307: (1.72), 308: (1.52), 309: (1.18), 310: (1.19), 311: (1.19), 312: (1.31), 313: (1.07), 314: (1.10), 315: (1.08), 316: (1.08), 317: (1.13), 318: (1.12), 319: (1.09), 320: (1.09), 321: (1.16), 322: (1.16), 323: (1.09), 324: (1.19), 325: (1.31), 326: (1.37), 327: (1.08), 328: (1.27), 329: (1.59), 330: (1.61), 331: (1.14), 332: (1.19), 333: (1.32), 334: (1.10), 335: (1.13), 336: (1.09), 337: (1.08), 338: (1.10), 339: (1.14), 340: (1.10), 341: (1.08), 342: (1.15), 343: (1.17), 344: (1.28), 345: (1.12), 346: (1.16), 347: (1.14), 348: (1.05), 349: (1.05), 350: (1.05), 351: (1.05), 352: (1.05), 353: (1.05), 354: (1.05), 355: (1.05), 356: (1.05), 357: (1.05), 358: (1.05), 359: (1.05), 360: (1.05), 361: (1.05), 362: (1.05), 363: (1.05), 364: (1.05), 365: (1.05), 366: (1.05), 367: (1.05), 368: (1.05), 369: (1.05), 370: (1.05), 371: (1.05), 372: (1.05), 373: (1.05)"#@param {type:"string"}

camera_preset_name = camera_preset
camera_preset = camera_presets[camera_preset]

translation_x = camera_preset["translation_x"]
translation_y = camera_preset["translation_y"]
translation_z = camera_preset["translation_z"]
rotation_3d_x = camera_preset["rotation_3d_x"]
rotation_3d_y = camera_preset["rotation_3d_y"]
rotation_3d_z = camera_preset["rotation_3d_z"]

midas_depth_model = "dpt_large"#@param {type:"string"}
midas_weight = 0.3#@param {type:"number"}
near_plane = 200#@param {type:"number"}
far_plane = 10000#@param {type:"number"}
fov = 120 #@param {type:"number"}
padding_mode = 'border'#@param {type:"string"}
sampling_mode = 'bicubic'#@param {type:"string"}

max_frames = frames



turbo_mode = True
turbo_steps = "5" #@param ["1","2","3","4","5","6"] {type:'string'}

turbo_steps = int(turbo_steps)

In [ ]:
def parse_key_frames(string, prompt_parser=None):
    """Given a string representing frame numbers paired with parameter values at that frame,
    return a dictionary with the frame numbers as keys and the parameter values as the values.

    Parameters
    ----------
    string: string
        Frame numbers paired with parameter values at that frame number, in the format
        'framenumber1: (parametervalues1), framenumber2: (parametervalues2), ...'
    prompt_parser: function or None, optional
        If provided, prompt_parser will be applied to each string of parameter values.
    
    Returns
    -------
    dict
        Frame numbers as keys, parameter values at that frame number as values

    Raises
    ------
    RuntimeError
        If the input string does not match the expected format.
    
    Examples
    --------
    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)")
    {10: 'Apple: 1| Orange: 0', 20: 'Apple: 0| Orange: 1| Peach: 1'}

    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)", prompt_parser=lambda x: x.lower()))
    {10: 'apple: 1| orange: 0', 20: 'apple: 0| orange: 1| peach: 1'}
    """
    import re
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param

    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames


def get_inbetweens(key_frames, integer=False):
    """Given a dict with frame numbers as keys and a parameter value as values,
    return a pandas Series containing the value of the parameter at every frame from 0 to max_frames.
    Any values not provided in the input dict are calculated by linear interpolation between
    the values of the previous and next provided frames. If there is no previous provided frame, then
    the value is equal to the value of the next provided frame, or if there is no next provided frame,
    then the value is equal to the value of the previous provided frame. If no frames are provided,
    all frame values are NaN.

    Parameters
    ----------
    key_frames: dict
        A dict with integer frame numbers as keys and numerical values of a particular parameter as values.
    integer: Bool, optional
        If True, the values of the output series are converted to integers.
        Otherwise, the values are floats.
    
    Returns
    -------
    pd.Series
        A Series with length max_frames representing the parameter values for each frame.
    
    Examples
    --------
    >>> max_frames = 5
    >>> get_inbetweens({1: 5, 3: 6})
    0    5.0
    1    5.0
    2    5.5
    3    6.0
    4    6.0
    dtype: float64

    >>> get_inbetweens({1: 5, 3: 6}, integer=True)
    0    5
    1    5
    2    5
    3    6
    4    6
    dtype: int64
    """
    key_frame_series = pd.Series([np.nan for a in range(max_frames)])

    for i, value in key_frames.items():
        key_frame_series[i] = value
    key_frame_series = key_frame_series.astype(float)
    
    interp_method = interp_spline

    if interp_method == 'Cubic' and len(key_frames.items()) <=3:
      interp_method = 'Quadratic'
    
    if interp_method == 'Quadratic' and len(key_frames.items()) <= 2:
      interp_method = 'Linear'
      
    
    key_frame_series[0] = key_frame_series[key_frame_series.first_valid_index()]
    key_frame_series[max_frames-1] = key_frame_series[key_frame_series.last_valid_index()]
    # key_frame_series = key_frame_series.interpolate(method=intrp_method,order=1, limit_direction='both')
    key_frame_series = key_frame_series.interpolate(method=interp_method.lower(),limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series


In [ ]:
import numpy as np

try:
    angle_series = get_inbetweens(parse_key_frames(angle))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `angle` correctly for key frames.\n"
        "Attempting to interpret `angle` as "
        f'"0: ({angle})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    angle = f"0: ({angle})"
    angle_series = get_inbetweens(parse_key_frames(angle))

try:
    zoom_series = get_inbetweens(parse_key_frames(zoom))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `zoom` correctly for key frames.\n"
        "Attempting to interpret `zoom` as "
        f'"0: ({zoom})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    zoom = f"0: ({zoom})"
    zoom_series = get_inbetweens(parse_key_frames(zoom))

try:
    translation_x_series = get_inbetweens(parse_key_frames(translation_x))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `translation_x` correctly for key frames.\n"
        "Attempting to interpret `translation_x` as "
        f'"0: ({translation_x})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    translation_x = f"0: ({translation_x})"
    translation_x_series = get_inbetweens(parse_key_frames(translation_x))

try:
    translation_y_series = get_inbetweens(parse_key_frames(translation_y))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `translation_y` correctly for key frames.\n"
        "Attempting to interpret `translation_y` as "
        f'"0: ({translation_y})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    translation_y = f"0: ({translation_y})"
    translation_y_series = get_inbetweens(parse_key_frames(translation_y))

try:
    translation_z_series = get_inbetweens(parse_key_frames(translation_z))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `translation_z` correctly for key frames.\n"
        "Attempting to interpret `translation_z` as "
        f'"0: ({translation_z})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    translation_z = f"0: ({translation_z})"
    translation_z_series = get_inbetweens(parse_key_frames(translation_z))

try:
    rotation_3d_x_series = get_inbetweens(parse_key_frames(rotation_3d_x))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `rotation_3d_x` correctly for key frames.\n"
        "Attempting to interpret `rotation_3d_x` as "
        f'"0: ({rotation_3d_x})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    rotation_3d_x = f"0: ({rotation_3d_x})"
    rotation_3d_x_series = get_inbetweens(parse_key_frames(rotation_3d_x))

try:
    rotation_3d_y_series = get_inbetweens(parse_key_frames(rotation_3d_y))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `rotation_3d_y` correctly for key frames.\n"
        "Attempting to interpret `rotation_3d_y` as "
        f'"0: ({rotation_3d_y})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    rotation_3d_y = f"0: ({rotation_3d_y})"
    rotation_3d_y_series = get_inbetweens(parse_key_frames(rotation_3d_y))

try:
    rotation_3d_z_series = get_inbetweens(parse_key_frames(rotation_3d_z))
except RuntimeError as e:
    print(
        "WARNING: You have selected to use key frames, but you have not "
        "formatted `rotation_3d_z` correctly for key frames.\n"
        "Attempting to interpret `rotation_3d_z` as "
        f'"0: ({rotation_3d_z})"\n'
        "Please read the instructions to find out how to use key frames "
        "correctly.\n"
    )
    rotation_3d_z = f"0: ({rotation_3d_z})"
    rotation_3d_z_series = get_inbetweens(parse_key_frames(rotation_3d_z))


In [ ]:
### 1.4 Define Midas functions
!pip install timm

from midas.dpt_depth import DPTDepthModel
from midas.midas_net import MidasNet
from midas.midas_net_custom import MidasNet_small
from midas.transforms import Resize, NormalizeImage, PrepareForNet
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2

# Initialize MiDaS depth model.
# It remains resident in VRAM and likely takes around 2GB VRAM.
# You could instead initialize it for each frame (and free it after each frame) to save VRAM.. but initializing it is slow.
default_models = {
    "midas_v21_small": f"{model_path}/midas_v21_small-70d6b9c8.pt",
    "midas_v21": f"{model_path}/midas_v21-f6b98070.pt",
    "dpt_large": f"{model_path}/dpt_large-midas-2f21e586.pt",
    "dpt_hybrid": f"{model_path}/dpt_hybrid-midas-501f0c75.pt",
    "dpt_hybrid_nyu": f"{model_path}/dpt_hybrid_nyu-2ce69ec7.pt",}


def init_midas_depth_model(midas_model_type="dpt_large", optimize=True):
    midas_model = None
    net_w = None
    net_h = None
    resize_mode = None
    normalization = None

    print(f"Initializing MiDaS '{midas_model_type}' depth model...")
    # load network
    midas_model_path = default_models[midas_model_type]

    if midas_model_type == "dpt_large": # DPT-Large
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitl16_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode = "minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "dpt_hybrid": #DPT-Hybrid
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitb_rn50_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode="minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "dpt_hybrid_nyu": #DPT-Hybrid-NYU
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitb_rn50_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode="minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "midas_v21":
        midas_model = MidasNet(midas_model_path, non_negative=True)
        net_w, net_h = 384, 384
        resize_mode="upper_bound"
        normalization = NormalizeImage(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    elif midas_model_type == "midas_v21_small":
        midas_model = MidasNet_small(midas_model_path, features=64, backbone="efficientnet_lite3", exportable=True, non_negative=True, blocks={'expand': True})
        net_w, net_h = 256, 256
        resize_mode="upper_bound"
        normalization = NormalizeImage(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    else:
        print(f"midas_model_type '{midas_model_type}' not implemented")
        assert False

    midas_transform = T.Compose(
        [
            Resize(
                net_w,
                net_h,
                resize_target=None,
                keep_aspect_ratio=True,
                ensure_multiple_of=32,
                resize_method=resize_mode,
                image_interpolation_method=cv2.INTER_CUBIC,
            ),
            normalization,
            PrepareForNet(),
        ]
    )

    midas_model.eval()
    
    if optimize==True:
        if device == torch.device("cuda"):
            midas_model = midas_model.to(memory_format=torch.channels_last)  
            midas_model = midas_model.half()

    midas_model.to(device)

    print(f"MiDaS '{midas_model_type}' depth model initialized.")
    return midas_model, midas_transform, net_w, net_h, resize_mode, normalization



In [ ]:

# Delete memory from previous runs
!nvidia-smi -caa
for var in ['device', 'model', 'perceptor', 'z']:
  try:
      del globals()[var]
  except:
      pass

try:
    import gc
    gc.collect()
except:
    pass

try:
    torch.cuda.empty_cache()
except:
    pass

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
midas_model, midas_transform, midas_net_w, midas_net_h, midas_resize_mode, midas_normalization = init_midas_depth_model(midas_depth_model)


# Troubleshooting
The most common cause of problems is trying to use a resolution that's too high, which uses up too much memory. Unfortunately the amount of memory Google Colab gives is variable depending on the session, so I can't give an exact figure for how high a resolution you can use. If you are having problems though, try reducing the resolution to 400x400 px, and increasing from there.
Two gotchas:
1. if you use an input image, this overwrites the resolution you specify.
2. You may be able to generate images, but then run into problems at the video generation stage. This is probably also due to a resolution that is too high, because all the memory was used at the image generation stage. If you save the images (if you are using google drive, they will already be saved), restart the kernel completely (Runtime > Manage Sessions > Terminate) and then start again at the video generation stage, you may be able to overcome this.

If you still have problems, then:

# Reporting problems
Submit bug reports or feature requests at https://github.com/chigozienri/VQGAN-CLIP-animations/issues
or ask me for help on Twitter at https://twitter.com/chigozienri

# How to use this notebook

This is an example of a Jupyter Notebook, running in Google Colab

It runs Python code in your browser. It's not hard to use, even if you haven't run code before.

First, in the menu bar, click Runtime>Change Runtime Type, and ensure that under "Hardware Accelerator" it says "GPU". If not, choose "GPU" from the drop-down menu, and click Save.

Then, run each of the cells in the notebook, one by one. Make sure to run all of them in order! Click in the cell, and press Shift-Enter on your keyboard. This will run the code in the cell, and then move to the next cell.

Follow the instructions in each cell, and you'll have an AI image in no time!

# Load Google Drive

Long-running colab notebooks might halt, and discard all progress. For this reason, it's useful (although optional) to save the images as they are produced in your personal google drive. Run the cell below to load google drive, click the link, sign in, paste the code generated into the prompt, and press enter. If you choose not to use google drive, uncheck the box below, but you should still run the cell

In [ ]:
#@markdown #**Run me**: Choose google drive or local connection 
from google.colab import drive

google_drive = False #@param {type:"boolean"}

if google_drive:
    drive.mount('/content/gdrive')
    working_dir = '/content/gdrive/MyDrive/vqgan'
else:
    working_dir = '/content'

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@markdown #**Check GPU type**
#@markdown ### Factory reset runtime if you don't have the desired GPU.

#@markdown ---




#@markdown V100 = Excellent (*Available only for Colab Pro users*)

#@markdown P100 = Very Good (*Available only for Colab Pro users*)

#@markdown T4 = Good (*Available only for Colab Pro users*)

#@markdown A100 = Not recommended, very slow (*Available only for Colab Pro users*)

#@markdown K80 = Good (*Default for non-Pro users*)

#@markdown P4 = (*Not Recommended*) 

#@markdown ---

gpu, = !nvidia-smi -L
print(gpu)

In [ ]:
#@title Setup for A100
#@markdown Running this cell might fix the problem with A100 GPUs being extremely slow. (untested)
if 'A100' in gpu:
  torch.backends.cudnn.enabled = False
  print('Finished setup for A100')
  

In [ ]:
#@markdown #**Run me**: Installation and loading of libraries and definitions
# @markdown This cell will take a while because you have to download multiple libraries

print("Downloading CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null

print("Downloading Python AI libraries...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
 
print("Installing libraries for handling metadata...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Installing Python video creation libraries...")
!pip install imageio-ffmpeg &> /dev/null
path = f'{working_dir}/steps'
!mkdir --parents {path}
print("Installation finished.")

import argparse
import math
from pathlib import Path
import sys
import os
import cv2
import pandas as pd
import numpy as np
import subprocess
import ast
 
sys.path.append('./taming-transformers')

# Some models include transformers, others need explicit pip install

!pip install transformers==4.19.2
import transformers

from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadata 
from libxmp import *         # metadata
import libxmp                # metadata
from stegano import lsb
import json
ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
 
def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    try:
        return vals[0], float(vals[1]), float(vals[2])
    except:
        return prompt, 1, float('-inf')
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

## Instructions for setting parameters:

If you want to create an animation with lots of keyframes, try out this spreadsheet by @EphemeralInc for constructing the strings: https://docs.google.com/spreadsheets/d/1sJ0PMHUPIYkS7LSxhzTThEP7rZ5CFonz-dBxqe8F2uc.

You can also try https://keyframe-string-generator.glitch.me/ or https://audio-keyframe-generator.glitch.me/ to construct strings using a visual editor, or an audio file.

| Parameter  |  Usage |
|---|---|
| `key_frames` | Whether to use key frames to change the parameters over the course of the run |
|  `text_prompts` |  Text prompts, separated by "\|" |
| `width` | Width of the output, in pixels. This will be rounded down to a multiple of 16 |
| `height` | Height of the output, in pixels. This will be rounded down to a multiple of 16 |
| `model` | Choice of model, must be downloaded above |
| `interval` | How often to display the frame in the notebook (doesn't affect the actual output) |
| `initial_image` | Image to start with (relative path to file) |
| `target_images` | Image prompts to target, separated by a pipe character ("\|") (relative path to files) |
| `seed` | Random seed, if set to a positive integer the run will be repeatable (get the same output for the same input each time, if set to -1 a random seed will be used. |
| `max_frames` | Number of frames for the animation |
| `angle` | Angle in degrees to rotate clockwise between each frame |
| `zoom` | Factor to zoom in each frame, 1 is no zoom, less than 1 is zoom out, more than 1 is zoom in (negative is uninteresting, just adds an extra 180 rotation beyond that in angle) |
| `translation_x` | Number of pixels to shift right each frame |
| `translation_y` | Number of pixels to shift down each frame |
| `iterations_per_frame` | Number of times to run the VQGAN+CLIP method each frame |
| `save_all_iterations` | Debugging, set False in normal operation |

---------

Transformations (zoom, rotation, and translation)

On each frame, the network restarts, is fed a version of the output zoomed in by `zoom` as the initial image, rotated clockwise by `angle` degrees, translated horizontally by `translation_x` pixels, and translated vertically by `translation_y` pixels. Then it runs `iterations_per_frame` iterations of the VQGAN+CLIP method. 0 `iterations_per_frame` is supported, to help test out the transformations without changing the image.

For `iterations_per_frame = 1` (recommended for more abstract effects), the resulting images will not have much to do with the prompts, but at least one prompt is still required.

In normal use, only the last iteration of each frame will be saved, but for trouble-shooting you can set `save_all_iterations` to True, and every iteration of each frame will be saved.

----------------

Mainly what you will have to modify will be `text_prompts`: there you can place the prompt(s) you want to generate (separated with |). It is a list because you can put more than one text, and so the AI tries to 'mix' the images, giving the same priority to both texts. You can also assign weights, to bias the priority towards one prompt or another, or negative weights, to remove an element (for example, a colour).

Example of weights with decimals:

Text : rubber:0.5 | rainbow:0.5

To use an initial image to the model, you just have to upload a file to the Colab environment (in the section on the left), and then modify `initial_image`: putting the exact name of the file. Example: sample.png

You can also change the model by changing the line that says `model`. Currently 1024, 16384, WikiArt, S-FLCKR and COCO-Stuff are available. To activate them you have to have downloaded them first, and then you can simply select it.

You can also use `target_images`, which is basically putting one or more images on it that the AI will take as a "target", fulfilling the same function as putting text on it. To put more than one you have to use | as a separator.

------------

Key Frames


## Note: this key frame format has changed. The old format will still work.

If `key_frames` is set to True, you are able to change the parameters over the course of the run.
To do this, put the parameters in in the following format:
`10: 0.5, 20: 1.0, 35: -1.0`

This means at frame 10, the value should be 0.5, at frame 20 the value should be 1.0, and at frame 35 the value should be -1.0. The value at each other frame will be linearly interpolated (that is, before frame 10, the value will be 0.5, between frame 10 and 20 the value will increase frame-by-frame from 0.5 to 1.0, between frame 20 and 35 the value will decrease frame-by-frame from 1.0 to -1.0, and after frame 35 the value will be -1.0)

This also works for text_prompts, e.g. `'Apple': {10: 1, 20: 0}, 'Orange': {10: 0, 20: 1}, 'Peach': {20: 1}`
will start with an Apple value of 1, once it hits frame 10 it will start decreasing in in Apple and increasing in Orange until it hits frame 20. Note that Peach will have a value of 1 the whole time.

It will also work for target_images, e.g. `'init.jpg': {0: 1, 10: 0}, 'final.jpg': {0: 0, 10: 1}`

If `key_frames` is set to True, all of the parameters which can be key-framed must be entered in this format.

In [ ]:
#@title Parameters
# @markdown The first time you run this cell with a given model selected, it will take a while because it will download the model. After that, if you change parameters, it won't take as long.

model = "vqgan_imagenet_f16_16384" #@param ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_16384", "coco", "faceshq", "sflckr"]


# @markdown New: Want Bézier easing in your animations? Try out https://keyframe-string-generator.glitch.me/ to generate keyframe strings. Want to use an audio file to drive parameters? Try https://audio-keyframe-generator.glitch.me/
key_frames = True #@param {type:"boolean"}
save_all_iterations = False#@param {type:"boolean"}

# option -C - skips download if already exists
!curl -C - -L -o {model}.yaml -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 1024
!curl -C - -L -o {model}.ckpt -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1'  #ImageNet 1024

if initial_image != "":
    print(
        "WARNING: You have specified an initial image. Note that the image resolution "
        "will be inherited from this image, not whatever width and height you specified. "
        "If the initial image resolution is too high, this can result in out of memory errors."
    )
elif width * height > 160000:
    print(
        "WARNING: The width and height you have specified may be too high, in which case "
        "you will encounter out of memory errors either at the image generation stage or the "
        "video synthesis stage. If so, try reducing the resolution"
    )
model_names={
    "vqgan_imagenet_f16_16384": 'ImageNet 16384',
    "vqgan_imagenet_f16_1024":"ImageNet 1024", 
    "wikiart_1024":"WikiArt 1024",
    "wikiart_16384":"WikiArt 16384",
    "coco":"COCO-Stuff",
    "faceshq":"FacesHQ",
    "sflckr":"S-FLCKR"
}
model_name = model_names[model]

if seed == -1:
    seed = None


## Where did the curve editor go?
The visual curve editor has been moved to https://keyframe-string-generator.glitch.me/

The following cell deletes any frames already in the steps directory. Make sure you have saved any frames you want to keep from previous runs

In [ ]:
path = f'{working_dir}/steps'
!rm -r {path}
!mkdir --parents {path}

In [ ]:

args = argparse.Namespace(
    prompts=text_prompts,
    image_prompts=target_images,
    noise_prompt_seeds=[],
    noise_prompt_weights=[],
    size=[width, height],
    init_weight=0.,
    clip_model='ViT-B/32',
    vqgan_config=f'{model}.yaml',
    vqgan_checkpoint=f'{model}.ckpt',
    step_size=step_size,
    cutn=48,
    cut_pow=1.,
    display_freq=interval,
    seed=seed,
)

In [ ]:
#text_prompts.items()
target_images

In [ ]:


def split_prompts(prompts):
  if prompts is None:
    return None
  prompt_series = pd.Series([np.nan for a in range(max_frames)])
  for i, prompt in prompts.items():
    prompt_series[i] = prompt
  # prompt_series = prompt_series.astype(str)
  prompt_series = prompt_series.ffill().bfill()
  return prompt_series

text_prompts_series = split_prompts(text_prompts)
target_images_series = split_prompts(target_images)
iterations_per_frame_series = split_prompts(iterations_per_frame)

In [ ]:
# tool to allow following the furthest place in the depth map using rotation
def get_center_mass(depth_map, preset_name):
  center_mass = [0, 0]
  total_mass = 0
  if preset_name == "follow_closest":
    depth_map = 1.0 - depth_map
  for y in range(depth_map.shape[0]):
    for x in range(depth_map.shape[1]):
      center_mass[0] += y * depth_map[y,x]
      center_mass[1] += x * depth_map[y,x]
      total_mass += depth_map[y, x]
  
  center_mass = [center_mass[0] / total_mass / depth_map.shape[0] * 2 - 1, center_mass[1] / total_mass / depth_map.shape[1]  * 2 - 1]
  return center_mass


In [ ]:


!pip install wav2clip
!pip install librosa

In [ ]:

import wav2clip
import librosa

if audio_file is not None:
  audio_sample_rate = 16000
  audio_hop_sec = 1
  hop_frames = int(audio_hop_sec * audio_sample_rate)
  audioperceptor =  wav2clip.get_model(frame_length=hop_frames, hop_length=hop_frames)

  def get_audio_embeddings(audio_prompt, fps):
    audio, sr = librosa.load(audio_prompt, sr=audio_sample_rate)
    #embeddings = wav2clip.embed_audio(audio, audioperceptor)
    embed = torch.from_numpy(wav2clip.embed_audio(audio, audioperceptor)).to(device)
    print(embed.shape)
    
    audio_prompts = []

    audio_amplitudes = []
    for i in range(int(embed.shape[2]*audio_hop_sec*fps)):
      embed_index = int(i / (audio_hop_sec * fps))
      #print("embed_index", i, embed_index)
      audio_prompts += [embed[:,:,embed_index]]

      audio_offset_start = int(sr * i / fps)
      audio_offset_end = int(sr * (i+1) / fps)
      amplitude =  np.log(1+np.abs(audio[audio_offset_start:audio_offset_end])*100).max()/ np.log(10)
      audio_amplitudes += [amplitude]

    audio_amplitudes = (audio_amplitudes - np.min(audio_amplitudes)) / (np.max(audio_amplitudes) - np.min(audio_amplitudes))
    print("audio prompts", len(audio_prompts))
    print("audio amplitudes", audio_amplitudes)
    # for i, embedding in enumerate(embed):
    #   frame = int(i * audio_hop_sec * fps)
    #   audio_prompts[frame] = embedding
    return audio_prompts, audio_amplitudes

  audio_embeddings, audio_amplitudes = get_audio_embeddings(audio_file, fps)

  max_frames = min(len(audio_amplitudes), max_frames)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

import librosa
#from google.colab.patches import cv2_imshow

img_0 = None
print('Using device:', device)
if not key_frames:
    if text_prompts:
        print('Using text prompts:', text_prompts)
    if target_images:
        print('Using image prompts:', target_images)
if args.seed is None:
    seed = torch.seed()
else:
    seed = args.seed
torch.manual_seed(seed)
print('Using seed:', seed)
 
model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)


cut_size = perceptor.visual.input_resolution
e_dim = model.quantize.e_dim
f = 2**(model.decoder.num_resolutions - 1)
make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
n_toks = model.quantize.n_e
toksX, toksY = args.size[0] // f, args.size[1] // f
sideX, sideY = toksX * f, toksY * f
z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]
stop_on_next_loop = False  # Make sure GPU memory doesn't get corrupted from cancelling the run mid-way through, allow a full frame to complete

def read_image_workaround(path):
    """OpenCV reads images as BGR, Pillow saves them as RGB. Work around
    this incompatibility to avoid colour inversions."""
    im_tmp = cv2.imread(path)
    return cv2.cvtColor(im_tmp, cv2.COLOR_BGR2RGB)

depth_map = None

for i in tqdm(range(max_frames)):
    print("-------- iteration",i )
    if stop_on_next_loop:
      break
    if key_frames:
        text_prompts = text_prompts_series[i]
        #text_prompts = [phrase.strip() for phrase in text_prompts.split("|")]
        if text_prompts == ['']:
            text_prompts = []
        args.prompts = text_prompts

        if target_images_series is not None:
          target_images = target_images_series[i]
          print("target images", target_images)
          if target_images == "None" or not target_images:
              target_images = []
          else:
              #target_images = target_images.split("|")
              target_images = [image.strip() for image in target_images]
        else:
          target_images = []
        args.image_prompts = target_images

        angle = angle_series[i]
        zoom = zoom_series[i]
        translation_x = translation_x_series[i]
        translation_y = translation_y_series[i]
        iterations_per_frame = iterations_per_frame_series[i]
        print(
            f'text_prompts: {text_prompts}',
            f'image_prompts: {target_images}',
            f'angle: {angle}',
            f'zoom: {zoom}',
            f'translation_x: {translation_x}',
            f'translation_y: {translation_y}',
            f'iterations_per_frame: {iterations_per_frame}'
        )
    try:
        if i == 0 and initial_image != "":
            img_0 = read_image_workaround(initial_image)
            z, *_ = model.encode(TF.to_tensor(img_0).to(device).unsqueeze(0) * 2 - 1)
        elif i == 0 and not os.path.isfile(f'{working_dir}/steps/{i:04d}.png'):
            one_hot = F.one_hot(
                torch.randint(n_toks, [toksY * toksX], device=device), n_toks
            ).float()
            z = one_hot @ model.quantize.embedding.weight
            z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
    
        # 3D stuff

        angle = angle_series[i]
        #zoom = args.zoom_series[frame_num]
        translation_x = translation_x_series[i]
        translation_y = translation_y_series[i]
        translation_z = translation_z_series[i]
        rotation_3d_x = rotation_3d_x_series[i]
        rotation_3d_y = rotation_3d_y_series[i]
        rotation_3d_z = rotation_3d_z_series[i]
        print(
            f'angle: {angle}',
            #f'zoom: {zoom}',
            f'translation_x: {translation_x}',
            f'translation_y: {translation_y}',
            f'translation_z: {translation_z}',
            f'rotation_3d_x: {rotation_3d_x}',
            f'rotation_3d_y: {rotation_3d_y}',
            f'rotation_3d_z: {rotation_3d_z}',
        )

        if i > 0:
          seed = seed + 1    
          img_filepath = '/content/progress.png'
          previous_filepath = '/content/previous.png'
          trans_scale = 1.0/200.0

          if (camera_preset_name == "follow_horizon"  or camera_preset_name == "follow_closest") and (depth_map is not None):
            center_mass = get_center_mass(depth_map, camera_preset_name)
            rotation_3d_x = (-1.0 * center_mass[0]) * 0.2
            rotation_3d_y = center_mass[1] * 0.2
            if audio_file is not None:
              translation_z = audio_amplitudes[i] * 14
            print("center_mass", center_mass,"translation_z", translation_z)
            #cv2_imshow(depth_map)
          translate_xyz = [-translation_x*trans_scale, translation_y*trans_scale, -translation_z*trans_scale]

          rotate_xyz = [rotation_3d_x, rotation_3d_y, rotation_3d_z]
          #print('translation:',translate_xyz)
          #print('rotation:',rotate_xyz)
          rot_mat = p3dT.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
          #print("rot_mat: " + str(rot_mat))
          img_new, depth_map = dxf.transform_image_3d(img_filepath, midas_model, midas_transform, device,
                                                 rot_mat, translate_xyz, near_plane, far_plane,
                                                 fov, padding_mode=padding_mode,
                                                 sampling_mode=sampling_mode, midas_weight=midas_weight, return_depth_map=True)
          img_old_transformed = None
          if os.path.exists(previous_filepath):
            img_old_transformed = dxf.transform_image_3d(previous_filepath, midas_model, midas_transform, device,
                                                 rot_mat, translate_xyz, near_plane, far_plane,
                                                 fov, padding_mode=padding_mode,
                                                 sampling_mode=sampling_mode, midas_weight=midas_weight)
            img_old_transformed.save(previous_filepath)


          if (img_old_transformed is not None) and turbo_steps > 1:
            blend_factor = (i % turbo_steps) / turbo_steps
            if blend_factor == 0:
              blend_factor = 1
            img_0 = Image.blend(img_old_transformed,img_new, blend_factor)
          else:
            img_0 = img_new
          save_output(i, img_0)
          img_new.save(img_filepath)
          if i % turbo_steps != 0:
            continue
          z, *_ = model.encode(TF.to_tensor(img_0).to(device).unsqueeze(0) * 2 - 1)

          
        #i += 1

        z_orig = z.clone()
        z.requires_grad_(True)
        opt = optim.Adam([z], lr=args.step_size)

        normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                        std=[0.26862954, 0.26130258, 0.27577711])

        pMs = []
        
        if audio_file is not None:
          embed = audio_embeddings[i]
          print("got audio embedding for frame", i, "embed")
          pMs.append(Prompt(embed).to(device))


        for prompt in args.prompts:
            txt, weight, stop = parse_prompt(prompt)
            embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()

            pMs.append(Prompt(embed, weight, stop).to(device))

        for prompt in args.image_prompts:
            path, weight, stop = parse_prompt(prompt)
            img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
            batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
            embed = perceptor.encode_image(normalize(batch)).float()
            pMs.append(Prompt(embed, weight, stop).to(device))
        
        #if img_0 is not None:
        #    batch = make_cutouts(TF.to_tensor(img_0).unsqueeze(0).to(device))
        #    embed = perceptor.encode_image(normalize(batch)).float()
        #    pMs.append(Prompt(embed, weight, stop).to(device))

        for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
            gen = torch.Generator().manual_seed(seed)
            embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
            pMs.append(Prompt(embed, weight).to(device))

        def synth(z):
            z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
            return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

        def add_xmp_data(filename):
            imagen = ImgTag(filename=filename)
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
            if args.prompts:
                imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
            else:
                imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', model_name, {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
            imagen.close()

        def add_stegano_data(filename):
            data = {
                "title": " | ".join(args.prompts) if args.prompts else None,
                "notebook": "VQGAN+CLIP",
                "i": i,
                "model": model_name,
                "seed": str(seed),
            }
            lsb.hide(filename, json.dumps(data)).save(filename)

        @torch.no_grad()
        def checkin(i, losses):
            losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
            tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
            !cp /content/progress.png /content/previous.png
            out = synth(z)
            TF.to_pil_image(out[0].cpu()).save('/content/progress.png')
            add_stegano_data('/content/progress.png')
            add_xmp_data('/content/progress.png')
            #display.display(display.Image('/content/progress.png'))
            preview_path = f"{output_path}/frame_{i:05d}.png"
            !cp /content/progress.png {preview_path}

        def save_output(i, img, suffix=None):
            filename = \
                f"{working_dir}/steps/{i:04}{'_' + suffix if suffix else ''}.png"
            imageio.imwrite(filename, np.array(img))
            add_stegano_data(filename)
            add_xmp_data(filename)

        def ascend_txt(i, save=True, suffix=None):
            out = synth(z)
            iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

            result = []

            if args.init_weight:
                result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

            for prompt in pMs:
                result.append(prompt(iii))
            img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
            img = np.transpose(img, (1, 2, 0))
            if i == 0:
                save_output(i, img, suffix=suffix)
            return result

        def train(i, save=True, suffix=None):
            opt.zero_grad()
            lossAll = ascend_txt(i, save=save, suffix=suffix)
            if i % args.display_freq == 0 and save:
                checkin(i, lossAll)
            loss = sum(lossAll)
            loss.backward()
            opt.step()
            with torch.no_grad():
                z.copy_(z.maximum(z_min).minimum(z_max))


            if iterations_per_frame == 0:
                save_output(i, img_0)
        j = 1

        while True:
            suffix = (str(j) if save_all_iterations else None)
            if j >= iterations_per_frame:
                train(i, save=True, suffix=suffix)
                break
            if save_all_iterations:
                train(i, save=True, suffix=suffix)
            else:
                train(i, save=False, suffix=suffix)
            j += 1

    except KeyboardInterrupt:
      stop_on_next_loop = True
      pass



## Make a video of the results

To generate a video with the frames, run the cell below. You can modify the number of FPS, the initial frame, the last frame, etc.

This step may fail due to an out-of-memory error.

In [ ]:

out_file=output_path+"/video.mp4"

!mkdir -p /tmp/ffmpeg
!cp /content/steps/*.png /tmp/ffmpeg
last_frame=!ls -t /tmp/ffmpeg/*.png | head -1
last_frame = last_frame[0]

# Copy last frame to start and duplicate at end so it sticks around 
!cp -v "{last_frame}" /tmp/ffmpeg/0000.png


encoding_options = "-c:v libx264 -crf 20 -preset slow -vf format=yuv420p -c:a aac -movflags +faststart"

!ffmpeg -r {fps} -i /tmp/ffmpeg/%*.png -y {encoding_options} /tmp/vid_no_audio.mp4
if audio_file is None:
  !ffmpeg -i /tmp/vid_no_audio.mp4 -f lavfi -i anullsrc -c:v copy -c:a aac -shortest -y "$out_file"
else:
  !ffmpeg -i /tmp/vid_no_audio.mp4 -i "{audio_file}" -c:v copy -c:a aac -shortest -y "$out_file"


print("Written", out_file)
!sleep 2
!rm -r /tmp/ffmpeg

In [ ]:
#plt.imshow(depth_map)

In [ ]:
print(output_path)
print(out_file)